In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import numpy
from sklearn import preprocessing
from sklearn.metrics import precision_recall_fscore_support, accuracy_score, confusion_matrix
from sklearn.ensemble import RandomForestClassifier

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
samples_path = "/content/drive/MyDrive/Gav/BolADRB3.xlsx"
df = pd.read_excel(samples_path)
df.head()

,Number,Allele1,Allele2,CD4,CD8,CD4CD8,CD21,SN,SP
0,72,*1201,*20012,8.96,9.83,0.9,7.53,0,1
1,56,*0101,*0101,50.80,20.52,2.5,0.50,0,1
2,96,*0601,*1101,31.20,38.40,0.8,5.04,1,0
3,95,*1101,*1802,15.59,20.90,0.7,14.57,0,1
4,94,*1201,*2704,49.95,16.36,3.1,7.25,1,0


In [ ]:
# scaler = preprocessing.MinMaxScaler()
# scaler = preprocessing.MaxAbsScaler()
# scaler = preprocessing.StandardScaler()
scaler = preprocessing.RobustScaler()
df.iloc[:,3:7]=scaler.fit_transform(df.iloc[:,3:7].to_numpy())
df = df.reindex(columns=['SN', 'SP', 'Allele1', 'Allele2', 'CD4', 'CD8', 'CD4CD8', 'CD21'])
df = df.join(
  pd.get_dummies(df.filter(like='Allele').stack())
    .groupby(level=0).max()
  ).drop(['Allele1', 'Allele2'], axis=1)
df.head()

,SN,SP,CD4,CD8,CD4CD8,CD21,*0101,*0201,*0301,*0303,...,*2709,*3001,*3201,*3202,*3401,*3501,*4201,*4301,*6801,*6901
0,0,1,-2.308367,-1.287911,-0.555556,0.482569,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,1,1.025498,-0.540881,1.222222,-0.807339,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,-0.536255,0.708595,-0.666667,0.025688,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,1,-1.780080,-0.514326,-0.777778,1.774312,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0.957769,-0.831586,1.888889,0.431193,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
X = df.iloc[:, 2:].values
Y = df.iloc[:, 1].values
x_train, x_test, y_train, y_test = train_test_split(X, Y,  test_size=0.2, random_state=42)

In [ ]:
clf = RandomForestClassifier(n_estimators = 10, criterion='log_loss', max_features=None, bootstrap=True, class_weight='balanced', random_state=2023)
clf.fit(x_train, y_train)
y_pred = clf.predict(x_test)
print(precision_recall_fscore_support(y_test, y_pred, average='binary'))
print(accuracy_score(y_test, y_pred))
print(confusion_matrix(y_test, y_pred)) # (tn, fp, fn, tp)

(0.6666666666666666, 0.4, 0.5, None)
0.84
[[19  1]
 [ 3  2]]


In [ ]:
cml = np.empty((2,2,2), int)
for srs in range(26, 46):
  x_train, x_test, y_train, y_test = train_test_split(X, Y,  test_size=0.2, random_state=srs)
  for crs in range(1924, 2024):
    clf = RandomForestClassifier(n_estimators = 10, criterion='log_loss', max_features=None, bootstrap=True, class_weight='balanced', random_state=crs)
    clf.fit(x_train, y_train)
    y_pred = clf.predict(x_test)
    cml = np.concatenate((cml, [confusion_matrix(y_test, y_pred)]), axis=0)
cml.mean(axis=0)

array([[16.44305694,  2.75774226],
       [ 4.51898102,  1.28021978]])

In [ ]:
features = pd.Series(clf.feature_importances_, index=df.iloc[:, 2:].columns)
features[features > 0.01]

CD4       0.265305
CD8       0.098693
CD4CD8    0.131320
CD21      0.192708
*0101     0.084827
*0902     0.040697
*1101     0.044701
*1201     0.023285
*1501     0.028125
*1506     0.010669
*1802     0.018616
dtype: float64